<a href="https://colab.research.google.com/github/DiegoeFarfan-SLA/labo2025v/blob/main/src/rpart/290_TareaHogar_02-C3GS-RF-NESTOR-PANIAGUA-1seeds_hijas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell
# ==========================
# Setup de buckets y dataset
# ==========================

# 1) Estructura de carpetas y symlink a buckets
mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -sf "/content/.drive/My Drive/labo1" /content/buckets/b1

# 2) Credenciales Kaggle (si las usás en otras celdas)
mkdir -p ~/.kaggle
if [ -f /content/buckets/b1/kaggle/kaggle.json ]; then
  cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
  chmod 600 ~/.kaggle/kaggle.json
fi

# 3) Estructuras internas
mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

# 4) Descarga dataset si no existe en bucket ni en /content
archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi

if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi

echo "✅ Setup de entorno y dataset listo."


✅ Setup de entorno y dataset listo.


limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657219,35.1,1454494,77.7,1326034,70.9
Vcells,1220161,9.4,8388608,64.0,1975136,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [3]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 304879
PARAM$qsemillas <- 16

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [4]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [5]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}


In [6]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}


In [7]:
# carpeta de trabajo
# por favor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900_304879_C4_MALDICION_GANADOR"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [8]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [9]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [10]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

REUSAR_CORRIDA_ANTERIOR <- FALSE

if( !REUSAR_CORRIDA_ANTERIOR ){
  if (file.exists("gridsearch_detalle.txt")) file.remove("gridsearch_detalle.txt")
  if (file.exists("gridsearch.txt"))          file.remove("gridsearch.txt")
}

# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla       = integer(),
    cp            = numeric(),
    maxdepth      = integer(),
    minsplit      = integer(),
    minbucket     = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] TRUE

[1] 0

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [11]:
library(data.table)

# ==============================
# GRID SEARCH *sin* reutilizar combos
# - Reanuda por combinación exacta (anti-join)
# - Mantiene rbind + fwrite como tu código
# ==============================

iter <- 0L

cp_vals        <- c(-0.10)     # 3 valores
maxdepth_vals  <- c(8)        # 5 valores
minsplit_vals  <- c(400)       # 4 valores
minbucket_vals <- c(30)       # 5 valores

for (vmax_depth in maxdepth_vals) {                       # loop externo = igual que tu código
  for (vcp in cp_vals) {                                  # ahora cp también barre
    for (vmin_split in minsplit_vals) {
      for (vmin_bucket in minbucket_vals) {

        # coherencia básica: si no se cumple, salteamos (esto deja 450 combinaciones)
        if (vmin_bucket > floor(vmin_split / 2)) next

        iter <- iter + 1
        cat(iter, " ")
        flush.console()

        # checkpoint para reanudar (idéntico al tuyo)
        if (iter * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) next

        # armo lista de hiperparámetros (idéntico a tu esquema)
        param_basicos <- list(
          "cp"        = vcp,          # complejidad (negativos permitidos como pediste)
          "maxdepth"  = vmax_depth,   # profundidad máxima
          "minsplit"  = vmin_split,   # mínimo de registros para split
          "minbucket" = vmin_bucket   # mínimo de registros en hoja
        )

        # corre Montecarlo con tus semillas y devuelve data.tables ya uniformadas
        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        # acumula resultados en la misma tabla de detalle (igual que tu código)
        tb_grid_search_detalle <- rbindlist(
          list(tb_grid_search_detalle, rbindlist(ganancias)),
          use.names = TRUE, fill = TRUE
        )
      }
    }
  }

  # grabar en el loop más externo (maxdepth), como en tu código original
  fwrite(tb_grid_search_detalle,
         file = "gridsearch_detalle.txt",
         sep = "\t")
}

cat("\nCombinaciones efectivas procesadas (≈): ", iter, "\n")




1  
Combinaciones efectivas procesadas (≈):  1 


In [12]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 16

In [13]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
828941,-0.1,8,400,30,58760000
176321,-0.1,8,400,30,40480000
675391,-0.1,8,400,30,48600000
785963,-0.1,8,400,30,51830000
174197,-0.1,8,400,30,48510000
674159,-0.1,8,400,30,46390000
935581,-0.1,8,400,30,49550000
370663,-0.1,8,400,30,47470000
472123,-0.1,8,400,30,50520000


In [14]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [15]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [16]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-0.1,8,400,30,47714375,16
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA


In [17]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Definí la ruta a tu archivo en Drive:
PATH_FILE = "/content/drive/MyDrive/labo1/exp/HT2900/gridsearch.txt"
# o, si querés usar el otro:
# PATH_FILE = "/content/drive/MyDrive/exp/HT2900/gridsearch_detalle.txt"


Mounted at /content/drive


In [ ]:
!pip install plotly pandas
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
from IPython.display import IFrame, HTML, display


In [ ]:
df = pd.read_csv(PATH_FILE, sep="\t")

# Aseguramos los tipos numéricos
for col in ["cp","maxdepth","minsplit","minbucket","ganancia_mean","qty"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Agregamos combinaciones (si hay múltiples corridas por semilla)
agg = (
    df.groupby(["cp","maxdepth","minsplit","minbucket"], as_index=False)
      .agg(ganancia_mean=("ganancia_mean","mean"),
           ganancia_sd=("ganancia_mean","std"),
           n_runs=("ganancia_mean","size"))
)

cp_levels    = sorted(agg["cp"].dropna().unique().tolist())
depth_levels = sorted(agg["maxdepth"].dropna().unique().tolist())

print(f"Archivo leído correctamente. Total de combinaciones: {len(agg)}")


Archivo leído correctamente. Total de combinaciones: 270


In [ ]:
# ===========================================
# Guardar en /content/drive/labo1/exp/HT2900/
# (o fallback a /content/drive/MyDrive/labo1/exp/HT2900/ si no existe)
# Mostrar inline + IFrame en Colab
# ===========================================
import os, glob, pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
from IPython.display import HTML, IFrame, display

# ---------- helpers ----------
def ensure_drive_mounted():
    # Colab crea /content/drive cuando el Drive está montado
    if not os.path.exists("/content/drive"):
        from google.colab import drive
        drive.mount("/content/drive")

def find_grid_file():
    # Usa PATH_FILE si está definido y existe
    if "PATH_FILE" in globals() and isinstance(PATH_FILE, str) and os.path.exists(PATH_FILE):
        print("Usando PATH_FILE definido:", PATH_FILE)
        return PATH_FILE
    # Busca en todo el drive y en /content
    patt = "gridsearch*.txt"
    cands = glob.glob("/content/drive/**/" + patt, recursive=True)
    cands += glob.glob("/content/" + patt, recursive=False)
    if not cands:
        raise FileNotFoundError("No encontré ningún gridsearch*.txt. Definí PATH_FILE o verificá el montaje de Drive.")
    cands.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    print("Usando archivo detectado:", cands[0])
    return cands[0]

def ensure_agg():
    global agg, cp_levels, depth_levels
    if "agg" in globals() and isinstance(agg, pd.DataFrame) and len(agg):
        return
    path = find_grid_file()
    df = pd.read_csv(path, sep="\t")
    for c in ["cp","maxdepth","minsplit","minbucket","ganancia_mean","qty"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    agg = (
        df.groupby(["cp","maxdepth","minsplit","minbucket"], as_index=False)
          .agg(ganancia_mean=("ganancia_mean","mean"),
               ganancia_sd=("ganancia_mean","std"),
               n_runs=("ganancia_mean","size"))
    ).sort_values(["cp","maxdepth","minsplit","minbucket"])
    cp_levels    = sorted(agg["cp"].dropna().unique().tolist())
    depth_levels = sorted(agg["maxdepth"].dropna().unique().tolist())
    print("Combinaciones agregadas:", len(agg))

def ensure_figs():
    global fig3d, parcoords
    # 3D
    if "fig3d" not in globals():
        traces = []
        for cpi in cp_levels:
            for di in depth_levels:
                sub = agg[(agg["cp"]==cpi) & (agg["maxdepth"]==di)]
                if sub.empty:
                    continue
                traces.append(go.Scatter3d(
                    x=sub["minsplit"], y=sub["minbucket"], z=sub["ganancia_mean"],
                    mode="markers", marker=dict(size=6),
                    name=f"cp={cpi} | depth={di}",
                    hovertemplate="<b>Ganancia</b>: %{z:.0f}<br>"
                                  "minsplit: %{x}<br>"
                                  "minbucket: %{y}<br>"
                                  f"cp: {cpi}<br>depth: {di}<extra></extra>"
                ))
        fig3d = go.Figure(traces)
        fig3d.update_layout(
            title="Grid Search — 3D interactivo (toggle por cp y depth)",
            scene=dict(xaxis_title="minsplit", yaxis_title="minbucket", zaxis_title="ganancia_mean"),
            height=650
        )
    # Parallel Coordinates
    if "parcoords" not in globals():
        parcoords = go.Figure([go.Parcoords(
            line=dict(color=agg["ganancia_mean"], showscale=True),
            dimensions=[
                dict(label="cp",        values=agg["cp"]),
                dict(label="depth",     values=agg["maxdepth"]),
                dict(label="minsplit",  values=agg["minsplit"]),
                dict(label="minbucket", values=agg["minbucket"]),
                dict(label="ganancia",  values=agg["ganancia_mean"])
            ]
        )])
        parcoords.update_layout(title="Grid Search — Parallel Coordinates", height=520)

def resolve_target_dir():
    """
    Intenta guardar en /content/drive/labo1/exp/HT2900/ exactamente.
    Si no existe ese árbol (porque el Drive real es MyDrive/Mi unidad/etc.),
    cae a /content/drive/MyDrive/labo1/exp/HT2900/ (lo crea).
    """
    requested = "/content/drive/labo1/exp/HT2900/"
    if os.path.isdir(requested):
        print("Guardando en:", requested)
        return requested
    # si no existe, probamos en MyDrive
    fallback = "/content/drive/MyDrive/labo1/exp/HT2900/"
    os.makedirs(fallback, exist_ok=True)
    print("Ruta solicitada no existe -> usando:", fallback)
    return fallback

# ---------- flujo ----------
ensure_drive_mounted()
ensure_agg()
ensure_figs()

# Crear HTML combinado (inline + IFrame)
html3d   = pio.to_html(fig3d, include_plotlyjs=False, full_html=False)
htmlpara = pio.to_html(parcoords, include_plotlyjs=False, full_html=False)
html_page = f"""
<!DOCTYPE html><html lang='es'><head>
<meta charset='utf-8'><title>Dashboard Grid Search</title>
<script src='https://cdn.plot.ly/plotly-latest.min.js'></script>
<style>body{{font-family:Arial,sans-serif;margin:20px}} .section{{margin-top:26px}}</style>
</head><body>
<h1>Dashboard — Grid Search</h1>
<p>3D: filtrá por <b>cp</b> desde el menú. En Parallel Coordinates, arrastrá rangos en los ejes.</p>
<div class="section"><h3>3D</h3>{html3d}</div>
<div class="section"><h3>Parallel Coordinates</h3>{htmlpara}</div>
</body></html>
"""

# Guardar primero en /content (para mostrar vía /files)
LOCAL_DASH = "/content/gridsearch_dashboard_interactivo.html"
with open(LOCAL_DASH, "w", encoding="utf-8") as f:
    f.write(html_page)

# Mostrar inline + IFrame
display(HTML("<h3>Dashboard (inline)</h3>"))
display(HTML(html_page))
display(HTML("<h3>Dashboard (IFrame desde /content)</h3>"))
display(IFrame(src=f"/files{LOCAL_DASH}", width="100%", height="640"))

# Guardar los 3 HTML también por separado
LOCAL_3D  = "/content/gridsearch_3d_interactivo.html"
LOCAL_PAR = "/content/gridsearch_parcoords_interactivo.html"
pio.write_html(fig3d,    file=LOCAL_3D,  include_plotlyjs="cdn", full_html=True)
pio.write_html(parcoords,file=LOCAL_PAR, include_plotlyjs="cdn", full_html=True)

# Copiar a la carpeta destino solicitada (o fallback a MyDrive si no existe)
TARGET_DIR = resolve_target_dir()
with open(os.path.join(TARGET_DIR, "gridsearch_dashboard_interactivo.html"), "w", encoding="utf-8") as f:
    f.write(html_page)
import shutil
shutil.copy2(LOCAL_3D,  os.path.join(TARGET_DIR, "gridsearch_3d_interactivo.html"))
shutil.copy2(LOCAL_PAR, os.path.join(TARGET_DIR, "gridsearch_parcoords_interactivo.html"))

print("\n✅ Guardado en:", TARGET_DIR)
print("   - gridsearch_dashboard_interactivo.html")
print("   - gridsearch_3d_interactivo.html")
print("   - gridsearch_parcoords_interactivo.html")


Ruta solicitada no existe -> usando: /content/drive/MyDrive/labo1/exp/HT2900/

✅ Guardado en: /content/drive/MyDrive/labo1/exp/HT2900/
   - gridsearch_dashboard_interactivo.html
   - gridsearch_3d_interactivo.html
   - gridsearch_parcoords_interactivo.html
